In [4]:
#Import Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from matplotlib import rcParams
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
#import seaborn as sns

from selenium import webdriver

print('Packages Imported')

Packages Imported


In [136]:
#Load in our CSV of Lego IDs taken from the other tool
ID_df_raw = pd.read_csv('SetIDs.csv', header=None)
#also going to remove any of the IDs that start with ISBN as these are books
ID_df_mod = ID_df_raw[ID_df_raw[0].str.contains("ISBN")==False]


In [162]:
ID_df_mod.describe()

,0
count,19145
unique,19145
top,700-12
freq,1


In [139]:
#selenium options
options = Options()

options.add_argument("--window-size=1920,1200")
options.add_argument('--headless')



In [142]:
#get text element from page
def get_textElement(driver,pageSource, text):
    return_text='null'
    if "<dt>"+text+"</dt>"in  pageSource:
        return_text = driver.find_element(By.XPATH, "//dt[text() = '"+text+"']/following-sibling::dd").text
    return return_text

In [175]:
#get dollar price
def get_price_dollars(driver,pageSource):
    price = 0
    if "<dt>RRP</dt>"in  pageSource:
        price_element = driver.find_element(By.XPATH, "//dt[text()= 'RRP']/following-sibling::dd")
        for i in price_element.text.split('/'):
            if '$' in i:
                price = i.strip().replace('$','')
    return price


In [177]:
#creating our dataframe also used for emtying the DF for testing
legoData_df = pd.DataFrame(columns=['Set number','Name','Set type','Theme','Theme group','Subtheme','Year released', 'Pieces','Minifigs','Price', 'Age range' ])
legoData_df

,Set number,Name,Set type,Theme,Theme group,Subtheme,Year released,Pieces,Minifigs,Price,Age range


In [178]:
driver = webdriver.Chrome(options=options)

for i,legoID in enumerate(ID_df_mod[0]):
    #legoID = ID_df_mod[0][0]
    #legoID ='215-3'
    driver.get('https://brickset.com/sets/'+str(legoID))
    pageSource = driver.page_source

    #individual fields
    lego_name = get_textElement(driver,pageSource,"Name")
    lego_setType = get_textElement(driver,pageSource,"Set type")
    lego_theme = get_textElement(driver,pageSource,"Theme")
    lego_themeGroup = get_textElement(driver,pageSource,"Theme group")
    lego_subtheme = get_textElement(driver,pageSource,"Subtheme")
    lego_yearReleased = get_textElement(driver,pageSource,"Year released")
    lego_pieces = get_textElement(driver,pageSource,"Pieces")
    #minifigs contains a bracket with how many of the minifigs are unique to this set. I'm exclusing this because as time
    #goes on more sets are released previoulsy unique minifigs become non unique. If I had "unique minifigs on release"
    #that could be an interesting feature
    lego_minifigs = get_textElement(driver,pageSource,"Minifigs").split(" ",1)[0]
    lego_price = get_price_dollars(driver,pageSource)
    lego_ageRange = get_textElement(driver,pageSource,"Age range")

    #adding to dataframe
    legoData_df.loc[len(legoData_df.index)] = [legoID,lego_name,lego_setType,lego_theme,lego_themeGroup,lego_subtheme, lego_yearReleased, 
                                     lego_pieces,lego_minifigs, lego_price, lego_ageRange]
    if i %100 ==0:
        print(i)
legoData_df

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

,Set number,Name,Set type,Theme,Theme group,Subtheme,Year released,Pieces,Minifigs,Price,Age range
0,700-12,Automatic Binding Bricks,Normal,SYSTEM,Basic,BASIC SET,1949,142,null,0,null
1,700_1_1-1,Individual 2 x 4 Bricks,Other,SYSTEM,Basic,SUPPLEMENTAL,1950,null,null,0,null
2,700_1_2-1,Individual 2 x 2 Bricks,Other,SYSTEM,Basic,SUPPLEMENTAL,1950,null,null,0,null
3,700_A-1,Automatic Binding Bricks Small Brick Set (Lego...,Other,SYSTEM,Basic,BASIC SET,1950,null,null,0,null
4,700_B_1-1,Individual 1 x 4 x 2 Window (without glass),Other,SYSTEM,Basic,SUPPLEMENTAL,1950,null,null,0,null
...,...,...,...,...,...,...,...,...,...,...,...
19140,5007851-1,"Brick Botanicals 1,000-Piece Puzzle",Gear,GEAR,Miscellaneous,JIGSAW,2023,null,null,22.99,9+
19141,BLOCKS099-1,Blocks magazine issue 99,Book,BOOKS,Miscellaneous,MAGAZINES/BLOCKS,2023,null,null,0,null
19142,BLOCKS100-1,Blocks magazine issue 100,Book,BOOKS,Miscellaneous,MAGAZINES/BLOCKS,2023,null,null,0,null
19143,BLOCKS101-1,Blocks magazine issue 101,Book,BOOKS,Miscellaneous,MAGAZINES/BLOCKS,2023,null,null,0,null


In [179]:
legoData_df.to_csv('legoData.csv', sep = ',')
